## Create a new project

It is always good idea to keep the projects seperate. This function creates a new project with subdirectories and a basic configuration file in the user defined directory otherwise the project is created in the current working directory.

You can always add new videos to the project at any stage of the project. 

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
import deeplabcut

In [9]:
import os
print(os.getcwd())
import matplotlib
%matplotlib inline

/home/gosztolai/Dropbox/github/fly_data_analysis/LiftFly3D/flylimbtracker


In [16]:
task='jointTracking' # Enter the name of your experiment Task
experimenter='PrismData' # Enter the name of the experimenter

videofile_path = [
    #"/data/LiftFly3D/prism/bottom_view/videos/video_191125_PR_Fly1_001_prism.mp4",
    #"/data/LiftFly3D/prism/bottom_view/videos/video_191125_PR_Fly1_002_prism.mp4",
    #"/data/LiftFly3D/prism/bottom_view/videos/video_191125_PR_Fly1_003_prism.mp4",
    #"/data/LiftFly3D/prism/bottom_view/videos/video_191125_PR_Fly1_004_prism.mp4",
    #"/data/LiftFly3D/prism/bottom_view/videos/video_191125_PR_Fly2_001_prism.mp4",
    #"/data/LiftFly3D/prism/bottom_view/videos/video_191125_PR_Fly2_002_prism.mp4",
    #"/data/LiftFly3D/prism/bottom_view/videos/video_191125_PR_Fly2_003_prism.mp4",
    #"/data/LiftFly3D/prism/bottom_view/videos/video_191125_PR_Fly2_004_prism.mp4",
    "/data/LiftFly3D/flylimbtracker/data/frames/cropped.mp4",
]

deeplabcut_dir = "/data/LiftFly3D/flylimbtracker/data/flylimbtracker_data/flylimbtracker-Adam Gosztolai-2020-08-18/"
config_path = deeplabcut_dir + "config.yaml"

In [14]:
deeplabcut.create_new_project(task, experimenter, videofile_path)

Project "/home/gosztolai/Dropbox/github/fly_data_analysis/LiftFly3D/flylimbtracker/jointTracking-PrismData-2020-08-18" already exists!


## Extract frames from videos 
A key point for a successful feature detector is to select diverse frames, which are typical for the behavior you study that should be labeled.

This function selects N frames either uniformly sampled from a particular video (or folder) (algo=='uniform'). Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.

Also make sure to get select data from different (behavioral) sessions and different animals if those vary substantially (to train an invariant feature detector).

Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later as well, it is advisable to crop the frames, to remove unnecessary parts of the frame as much as possible.

Always check the output of cropping. If you are happy with the results proceed to labeling.

In [ ]:
deeplabcut.extract_frames(config_path, 'automatic', 'kmeans', crop=False, userfeedback=True)
# there are other ways to grab frames, such as by clustering 'kmeans'; please see the paper.

## Label the extracted frames
Only videos in the config file can be used to extract the frames. Extracted labels for each video are stored in the project directory under the subdirectory **'labeled-data'**. Each subdirectory is named after the name of the video. The toolbox has a labeling toolbox which could be used for labeling. 

In [ ]:
%gui wx
deeplabcut.label_frames(config_path)

## Create a training dataset
This function generates the training data information for DeepCut (which requires a mat file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [17]:
deeplabcut.create_training_dataset(config_path)

/data/LiftFly3D/flylimbtracker/data/flylimbtracker_data/flylimbtracker-Adam Gosztolai-2020-08-18/training-datasets/iteration-0/UnaugmentedDataSet_flylimbtrackerAug18  already exists!
/data/LiftFly3D/flylimbtracker/data/flylimbtracker_data/flylimbtracker-Adam Gosztolai-2020-08-18/dlc-models/iteration-0/flylimbtrackerAug18-trainset95shuffle1  already exists!
/data/LiftFly3D/flylimbtracker/data/flylimbtracker_data/flylimbtracker-Adam Gosztolai-2020-08-18/dlc-models/iteration-0/flylimbtrackerAug18-trainset95shuffle1//train  already exists!
/data/LiftFly3D/flylimbtracker/data/flylimbtracker_data/flylimbtracker-Adam Gosztolai-2020-08-18/dlc-models/iteration-0/flylimbtrackerAug18-trainset95shuffle1//test  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


## Start training

This function trains the network for a specific shuffle of the training dataset. 

In [18]:
deeplabcut.train_network(config_path,
                         displayiters=2000,
                         saveiters=20000,
                         maxiters=200000)

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
                [20],
                [21],
                [22],
                [23],
                [24],
                [25],
                [26],
                [27],
                [28],
                [29]],
 'all_joints_names': ['body-coxa front L',
                      'coxa-femur front L',
                      'femur-tibia front L',
                      'tibia-tarsus front L',
                      'tarsus tip front L',
                      'body-coxa mid L',
                      'coxa-femur mid L',
                      'femur-tibia m

Switching batchsize to 1, as default/tensorpack/deterministic loaders do not support batches >1. Use imgaug loader.
Starting with standard pose-dataset loader.
Initializing ResNet
Loading ImageNet-pretrained resnet_50


InternalError: Failed to create session.

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [ ]:
deeplabcut.evaluate_network(config_path, plotting=True)

## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [ ]:
deeplabcut.analyze_videos(config_path,videofile_path, videotype='.mp4')

## Extract outlier frames [optional step]
This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. Make sure to provide the correct value of the "iterations" as it will be used to create the unique directory where the extracted frames will be saved.

In [ ]:
path_config_file = '/home/morales/Documents/Morales/fly_lane-DM-2019-02-19/config.yaml'
crop_video = '/home/morales/Documents/Morales/MDN3_Screen/GFP/190315/161441_s0a0_p75/output_40fps_5.avi'

In [ ]:
path_config_file = '/home/morales/Documents/Morales/fly_lane-DM-2019-02-19/config.yaml'
crop_video = '/home/morales/Documents/Morales/MDN3_Screen/GFP/190315/161441_s0a0_p75/output_40fps_5.avi'

deeplabcut.extract_outlier_frames(path_config_file,[crop_video])

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset.

In [ ]:
%gui wx
deeplabcut.refine_labels(path_config_file)

In [ ]:
#Once all folders are relabeled, check them and advance. See how to check labels, above!
deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset [optional step]
Following the refine labels, append these frames to the original dataset to create a new iteration of training dataset.

In [ ]:
deeplabcut.create_labeled_video(config_path,videofile_path, save_frames =True)

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
deeplabcut.create_labeled_video(config_path,videofile_path)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook 
deeplabcut.plot_trajectories(path_config_file,videofile_path)

In [ ]:
sess.close()